In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import string
import re
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
seed = random.randint(0, 2 ** 32 - 1)
seed = 1985442815
random.seed(seed)
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
# device_name = "cpu"
device = torch.device(device_name)
print(f"Will use {device_name} for training with seed: {seed}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M')

In [ ]:
data = []
with open("./sentiment_training_data.txt") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if len(line) > 0:
            line_split = line.split(" ")
            data.append({
                'dragon': line_split[0],
                'sentiment': line_split[1],
                'text': " ".join(line_split[2:])
            })
data